In [ ]:
from modules import *
%matplotlib widget

### 1. Load dataset

In [ ]:
datapicker = create_datapicker()
display(datapicker)

### 2. Explore Dataset Maps

In [ ]:
datapicker.geodata.columns

In [ ]:
exploremap(datapicker.geodata, column='empatica_e4_gsr')


### 3. Explore Dataset Streams

In [ ]:
WorldCamera = datapicker.dataset.streams.PupilLabs.WorldCamera.data
WorldCamera

In [ ]:
gaze_data = datapicker.dataset.streams.PupilLabs.Gaze.data
gaze_data.Timestamp.plot()
gaze_data

In [ ]:
gaze_counter = datapicker.dataset.streams.PupilLabs.Counter.Gaze
diff_stats(gaze_counter)
gaze_counter

In [ ]:
gaze_harp = gaze_counter.data.reset_index()
gaze = gaze_harp.join(gaze_data)
gaze.set_index('Seconds', inplace= True)
gaze

In [ ]:
plot_traces({
    'accelX': datapicker.dataset.streams.Accelerometer.data["Accl_X"],
    'temp (C)': datapicker.dataset.streams.TK.AirQuality.Temperature.data/100,
    'angleX': datapicker.dataset.streams.Accelerometer.data["Orientation_X"],
    'altitude': datapicker.dataset.georeference.elevation,
    'iaq': datapicker.dataset.streams.TK.AirQuality.IAQIndex.data,
    'e4': datapicker.dataset.streams.Empatica.data.E4_Gsr.Value,
    'pupil': gaze["GazeX"]
})

# plot_traces({
#     # 'accelX': datapicker.dataset.streams.Accelerometer.data["Accl_X"],
#     # 'temp (C)': datapicker.dataset.streams.TK.AirQuality.Temperature.data/100,
#     # 'angleX': datapicker.dataset.streams.Accelerometer.data["Orientation_X"],
#     # 'altitude': datapicker.dataset.georeference.elevation,
#     # 'iaq': datapicker.dataset.streams.TK.AirQuality.IAQIndex.data,
#     # 'e4': datapicker.dataset.streams.E4.Gsr,
# })

### 4. Export Dataset to OGC API

In [ ]:
record = DatasetRecord(datapicker.dataset, datapicker.geodata, properties=RecordProperties(
    title='<City> Outdoor Walk: <CityRegion> Subject <ID>',
    description='Outdoor walk data collection',
    license='CC BY-NC 4.0',
    tool='Bonsai',
    keywords=['<City>', 'Outdoor', 'Walk', 'Microclimate', 'Biosignals'],
    contacts=[
        Contact(
            name='Your Name',
            institution='Your Institution',
            email='youremail@yourdomain.com'
        )
    ],
    themes=[]
))

In [ ]:
rpath = Path(record.id)
export_geoframe_to_geojson(datapicker.geodata, rpath.with_suffix('.geojson'))
with open(rpath.with_suffix('.json'), 'w') as f:
    f.write(record.to_json())